In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt

In [ ]:
f = open("../input/flickr8k/captions.txt", "r")
images_names = []
caption = []
for x in f:
    a = x.split(",")
    images_names.append(a[0])
    caption.append(a[1])
    
images_names = images_names[1:]
caption = caption[1:]
print(images_names[0:10])
print(caption[0:10])

In [ ]:
image_cap_dict = {}
for i in range(len(images_names)):
    image_name = images_names[i]
    if image_name not in image_cap_dict.keys():
        image_cap_dict[image_name] = []
        image_cap_dict[image_name].append(caption[i])
    else:
        image_cap_dict[image_name].append(caption[i])

In [ ]:
for sentences in image_cap_dict.keys():
    new = []
    for sent in image_cap_dict[sentences]:
        a = sent.strip()
        new.append(a)
    image_cap_dict[sentences] = new    

In [ ]:
test = image_cap_dict["1000268201_693b08cb0e.jpg"]
print(image_cap_dict)

In [ ]:
#../input/flickr8k/Images/1000268201_693b08cb0e.jpg

images_path = "../input/flickr8k/Images"
image = "1012212859_01547e3f17.jpg"
path = images_path + "/" + image

from IPython.display import Image, display
z = Image(filename=path)
display(z)

for sent in image_cap_dict[image]:
    print(sent)

In [ ]:
len(image_cap_dict.keys())

In [ ]:
train_dataset = open('flickr_8k_train_dataset.txt','wb')
train_dataset.write(b"image_id\tcaptions\n")

test_dataset = open('flickr_8k_test_dataset.txt','wb')
test_dataset.write(b"image_id\tcaptions\n")

In [ ]:
a = [k for k in image_cap_dict.keys()]

In [ ]:
keys = image_cap_dict.keys()
#print(keys)
for img_name in a[0:6000]:
    list_cap = image_cap_dict[img_name]
    for capt in list_cap:
        caption = "<start> "+ capt + " <end>"
        train_dataset.write((img_name+"\t"+caption+"\n").encode())
        print((img_name+"\t"+caption+"\n").encode())
        train_dataset.flush()
train_dataset.close()

In [ ]:
ann = []
keys = image_cap_dict.keys()
for i in keys:
    ann.append(i)
    
print(ann[10])


In [ ]:
for img_name in a[6000:]:
    list_cap = image_cap_dict[img_name]
    for capt in list_cap:
        caption = "<start> "+ capt + " <end>"
        test_dataset.write((img_name+"\t"+caption+"\n").encode())
        print(img_name+"\t"+caption+"\n")
        test_dataset.flush()
test_dataset.close()

In [ ]:
f = open('flickr_8k_test_dataset.txt','r')
f.readlines()

In [ ]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
model.summary()

In [ ]:
# Helper function to process images
def preprocessing(img_path):
    im = tf.keras.preprocessing.image.load_img(img_path,target_size=(224,224,3))
    im = tf.keras.preprocessing.image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
import tensorflow as tf
x_train = a[0:6000]
train_data = {}
image_path = "../input/flickr8k/Images"
ctr=0
for image_name in x_train:
    if image_name == "":
        continue
    ctr+=1
    if ctr%100==0:
        print(ctr)
        print(pred)
    path = images_path + "/" + image_name
    img = preprocessing(path)
    pred = model.predict(img).reshape(2048)
    train_data[image_name] = pred

In [ ]:
train_data['101654506_8eb26cfb60.jpg'].shape

In [ ]:
with open( "trained_images_2.p", "wb" ) as pickle_f:
    pickle.dump(train_data, pickle_f ) 

In [ ]:
dataset = pd.read_csv("flickr_8k_train_dataset.txt", delimiter='\t')
ds = dataset.values
print(ds.shape)

In [ ]:
print(ds)

In [ ]:
dataset.tail(2)

In [ ]:
sentences = []
i = 0
for i in range(ds.shape[0]):
    sentences.append(ds[i, 1])
    if i%90==0:
        print(ds[i, 1])
    
print(len(sentences))

In [ ]:
sentences[0:10]

In [ ]:
words = [sentence.split() for sentence in sentences]
words[1]

In [ ]:
i = 0
all_words = []
for k in words:
    for word in k:
        all_words.append(word)
print(all_words[0:40])

In [ ]:
unique = list(set(all_words))
print(len(unique))
vocab_size = len(unique)

In [ ]:
word_2_indices = {val:index for index,val in enumerate(unique)}
indices_2_word = {index:val for index,val in enumerate(unique)}

In [ ]:
word_2_indices['UNK'] = 0
indices_2_word[0] = 'UNK'

In [ ]:
print(word_2_indices.keys())

In [ ]:
word_2_indices["skimpy"]

In [ ]:
indices_2_word[300]

In [ ]:
print(word_2_indices['<start>'])
print(indices_2_word[4090])
print(word_2_indices['<end>'])
print(indices_2_word[8251])

In [ ]:
vocab_size = len(word_2_indices.keys())
print(vocab_size)

In [ ]:
max_len = -1
for k in sentences:
    a = k.split(" ")
    
    if len(a)>max_len:
        max_len = len(a)
        arr = a
print(max_len)
print(arr)

In [ ]:
padded_sequences = []
subsequent_words = []

for i in range(ds.shape[0]):
    partial_seqs = []
    next_words = []
    
    text = ds[i,1].split(" ")
    text = [word_2_indices[i] for i in text]
    
    for i in range(0,len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')
    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)
   
padded_sequences = np.asarray(padded_sequences)
subsequent_words = np.asarray(subsequent_words)

print(padded_sequences.shape)
print(subsequent_words.shape)

In [ ]:
print(padded_sequences[0].shape)
print(padded_sequences[0])


In [ ]:
#20,36
for x in range(len(padded_sequences[0])):
    array_ = []
    for y in range(max_len):
        array_.append(indices_2_word[padded_sequences[0][x][y]])
    print(array_)
    print("\n")

In [ ]:
num_of_images = 2000
captions = np.zeros([0, max_len])
next_words = np.zeros([0, vocab_size])

In [ ]:
for i in range(num_of_images):
    captions = np.concatenate([captions, padded_sequences[i]])
    next_words = np.concatenate([next_words, subsequent_words[i]])
    if(i%100==0):
        print(i)

np.save("captions_2.npy", captions)
np.save("next_words_2.npy", next_words)

print(captions.shape)
print(next_words.shape)

In [ ]:
print(captions.shape)
print(padded_sequences[i].shape)

In [ ]:
with open('./trained_images_2.p', 'rb') as f:
    encoded_images = pickle.load(f)

In [ ]:
encoded_images

In [ ]:
imgs = []
for x in range(ds.shape[0]):
    if ds[x,0] in encoded_images.keys():
        imgs.append(list(encoded_images[ds[x, 0]]))
        
imgs = np.asarray(imgs)
print(imgs.shape)

In [ ]:
#num_of_images = 3600
images = []

for ix in range(num_of_images):
    for iy in range(padded_sequences[ix].shape[0]):
        images.append(imgs[ix])
        
images = np.asarray(images)

np.save("images_2.npy", images)

print(images.shape)

In [ ]:
image_names = []

for ix in range(num_of_images):
    for iy in range(padded_sequences[ix].shape[0]):
        image_names.append(ds[ix, 0])
        
image_names = np.asarray(image_names)

np.save("image_names_2.npy", image_names)

print(len(image_names))

In [ ]:
#MODEL

In [ ]:
captions = np.load("./captions_2.npy")
next_words = np.load("./next_words_2.npy")
images = np.load("./images_2.npy")
image_name = np.load("./image_names_2.npy")

In [ ]:
embedding_size = 128 
max_len = 36

In [ ]:
image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

In [ ]:
language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

In [ ]:
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(256, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

#model.load_weights("./model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit([images, captions], next_words, batch_size=512, epochs=100)

In [ ]:
model.save_weights("model_weights.h5")

In [ ]:
def preprocessing(img_path):
    im = tf.keras.preprocessing.image.load_img(img_path,target_size=(224,224,3))
    im = tf.keras.preprocessing.image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
keys = [k for k in image_cap_dict.keys()]

In [ ]:
len(keys)

In [ ]:
import tensorflow as tf
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

import random
n = random.randint(2000,8000)
m = keys[n]
#print(str(n) + "\t" + m)
img = "../input/flickr8k/Images" + "/" + m
test_img = get_encoding(resnet, img)
Argmax_Search = predict_captions(test_img)

z = Image(filename=img)
display(z)

#print(Argmax_Search)

In [ ]:
new_sentence = Argmax_Search
print(Argmax_Search)